In [8]:
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
#from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D,Dropout,SeparableConv2D,BatchNormalization, Activation, Dense
from keras.applications.vgg19 import VGG19
from keras.applications.resnet import ResNet101
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, Lambda, Dense, Flatten
import matplotlib.pyplot as plt

# dataset has 3 classes
num_class = 2

# Base model without Fully connected Layers
base_model = VGG19(include_top=False, weights='imagenet', input_shape=(224,224,3))

# don't train existing weights
for layer in base_model.layers:
  layer.trainable = False
  
  
x=base_model.output
x = Flatten()(base_model.output)
##3 classes PNEUNOMIA, NORMAL


preds=Dense(num_class, activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)
##check the model
model.summary()


# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


##Preparing Data 
train_datagen=ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input,
                                 validation_split=0.25)

train_generator=train_datagen.flow_from_directory(r"C:\Users\Computing\Downloads\data-task1\data-task1\train",
                                                 target_size=(224,224),
                                                 batch_size=64,
                                                 class_mode='categorical',
                                                 subset='training')


validation_generator = train_datagen.flow_from_directory(r"C:\Users\Computing\Downloads\data-task1\data-task1\train", # same directory as training data
                                                target_size=(224,224),
                                                batch_size=64,
                                                class_mode='categorical',
                                                subset='validation') # set as validation data



##Training
##SEtting hyper parameter
epochs = 50
learning_rate = 0.0005
decay_rate = learning_rate / epochs
opt = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, decay=decay_rate, amsgrad=False)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])


##Train
step_size_train = train_generator.n/train_generator.batch_size
step_size_val = validation_generator.samples // validation_generator.batch_size
r = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_data = validation_generator, 
                   validation_steps =step_size_val,
                   epochs=5)


##Confusion Matrix  (Sample code for confusion matrix but this is not working, it doesn't produce error neither results)
from sklearn.metrics import classification_report, confusion_matrix
batch_size=64
Y_pred = model.predict_generator(validation_generator,186)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['NORMAL','PNEUMONIA']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

<ipython-input-8-5091d35bec64>:78: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(generator=train_generator,


Epoch 1/5
35/35 [==============================] - 224s 6s/step - loss: 2.5416 - accuracy: 0.7610 - val_loss: 3.0392 - val_accuracy: 0.7884
Epoch 2/5
35/35 [==============================] - 227s 6s/step - loss: 0.8959 - accuracy: 0.8623 - val_loss: 2.3412 - val_accuracy: 0.7628
Epoch 3/5
35/35 [==============================] - 227s 6s/step - loss: 0.3260 - accuracy: 0.9311 - val_loss: 2.1611 - val_accuracy: 0.7955
Epoch 4/5
35/35 [==============================] - 225s 6s/step - loss: 0.1161 - accuracy: 0.9716 - val_loss: 2.0791 - val_accuracy: 0.7798
Epoch 5/5
35/35 [==============================] - 229s 7s/step - loss: 0.0732 - accuracy: 0.9742 - val_loss: 2.3088 - val_accuracy: 0.8026


<ipython-input-8-5091d35bec64>:88: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(validation_generator,186)


Confusion Matrix
[[514  75]
 [133  27]]
Classification Report
              precision    recall  f1-score   support

      NORMAL       0.79      0.87      0.83       589
   PNEUMONIA       0.26      0.17      0.21       160

    accuracy                           0.72       749
   macro avg       0.53      0.52      0.52       749
weighted avg       0.68      0.72      0.70       749

